**Modeling tumor growth with Genetic Algorithm**

This notebook uses the non-dimensionalized KP model.

Genetic algorithm:
DOI: https://doi-org.proxy.lib.ohio-state.edu/10.1103/PhysRevE.70.016211

Onco model:
DOI: https://doi.org/10.1371/journal.pone.0004271

Dixon et al.:
DOI: https://doi-org.proxy.lib.ohio-state.edu/10.1016/j.mbs.2024.109187

Sarv Ahrabi and Momenzadeh:
DOI: https://doi.org/10.1007/s00285-020-01525-7

Kirschner-Panetta model:
Kirschner, D., Panetta, J. Modeling immunotherapy of the tumor – immune interaction. J Math Biol 37, 235–252 (1998). https://doi.org/10.1007/s002850050127

In [1]:
# Main module
%reset -f

# autoreload imports
%load_ext autoreload
%autoreload 2

# import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
import scipy
from scipy.integrate import solve_ivp
import pygad
import csv
import tqdm
import time

# import model functions and Genetic Algorithm
from Model.KP_model import dx_dt, dy_dt, dz_dt, r_2, nondim, kp_coupled
from GA.fitness_function import GeneticAlgorithm
from Model.integration import kp_integrate


## Model with no treatment input

In [123]:
# Dimensional parameters
# Arrow indicates parameters that should be varied for different simulations

# eq.1
c_values = [0, 8.5e-5, 0.0025, 0.006, 0.010, 0.0297, 0.0325, 0.040, 0.050] # antigenicity values to test
c = 0.0297  # antigenicity (keep between 0 and 0.05) <---

mu_2 = 0.03 # Multiplicative inverse of the natural lifetime of effector cells (days^-1)
p_1 = 0.1245 # Proliferation rate of effector cells (days^-1) 
g_1 = 2 * 10**7 # Threshold growth rate of effector cells (cells days^-1)

# eq.2
g_2 = 1 * 10**5 # Threshold for tumor cell removal (cells)
r_2 = 0.18 # tumor growth rate (days^-1) <---
b = 1 * 10**(-9) # Multiplicative inverse of the tumor carrying capacity (cells^-1)
alpha = 1 # Immune system strength for tumor removal (days^-1)

# eq.3
mu_3 = 10 # Multiplicative inverse of the natural lifetime of IL-2 (days^-1)
p_2 = 5.0 # Production rate of IL-2 (units days^-1)
g_3 = 1 * 10**3 # Threshold for IL-2 production via effector-tumor interaction (cells)

# External therapy parameters
s_1 = 0  # external IL-2 source
s_2 = 0  # external T cell source

# Simulation parameters
num_steps = 2000 # number of time steps <---
total_time = 4000  # total time (days) <---

# time arrays
t_s = r_2  # time scale (days) <---
t = np.linspace(0, total_time, int(num_steps/r_2))  # time array
tau = t * t_s  # dimensional time array

# Arrays to hold non-dimensional values
x = np.zeros(num_steps)
y = np.zeros(num_steps)
z = np.zeros(num_steps)
s_1_array = np.zeros(num_steps)
s_2_array = np.zeros(num_steps)

# Arrays to hold population values
E = np.zeros(num_steps)
T = np.zeros(num_steps)
IL = np.zeros(num_steps)
IL_input = np.zeros(num_steps)

# Set initial conditions
E[0], T[0], IL[0] = 1e5, 1e5, 1e2

print("Nondimensionalized parameters:")
print([E[0], T[0], IL[0], t_s, c, p_1, g_1, mu_2, g_2, b, r_2, alpha, mu_3, p_2, g_3, s_1, s_2])

# Nondimensionalize parameters and initial conditions
[c, p_1, g_1, mu_2, g_2, b, r_2, alpha, mu_3, p_2, g_3, s_1, s_2] = nondim(
    E[0], T[0], IL[0], t_s, c, p_1, g_1, mu_2, g_2, b, r_2, alpha, mu_3, p_2, g_3, s_1, s_2)

print("Nondimensionalized parameters:")
print([c, p_1, g_1, mu_2, g_2, b, r_2, alpha, mu_3, p_2, g_3, s_1, s_2])

# Set initial non-dimensional x, y, z values to 1.0
x[0] = E[0] / E[0]
y[0] = T[0] / T[0]
z[0] = IL[0] / IL[0]

# Arrays to hold fitness values
fitness = np.zeros(num_steps)
fitness_history = []


Nondimensionalized parameters:
[100000.0, 100000.0, 100.0, 0.18, 0.0297, 0.1245, 20000000, 0.03, 100000, 1e-09, 0.18, 1, 10, 5.0, 1000, 0, 0]
Nondimensionalized parameters:
[0.165, 0.6916666666666667, 200000.0, 0.16666666666666666, 1.0, 0.0001, 1.0, 5.555555555555555, 55.55555555555556, 27777.777777777777, 0.01, 0.0, 0.0]


In [127]:
# Initialize KP integrator
integrator = kp_integrate()

# Time-stepping loop
for step in tqdm.tqdm(range(1, num_steps)):
   
    # External input parameters
    s_1_array[step] = 0
    s_2_array[step] = 0

    # parameters for time step
    params = [c, mu_2, p_1, g_1, s_1_array[step], r_2, b, alpha, g_2, p_2, g_3, mu_3, s_2_array[step]]

    # Solve KP ODEs for time step
    [x[step], y[step], z[step]] = integrator.integrate([x[step-1], y[step-1], z[step-1]], 
                                         params, 
                                         t_span=(tau[step-1], tau[step]))

    E[step] = x[step] * E[0]  # dimensionalize back to E
    T[step] = y[step] * T[0]  # dimensionalize back to T
    IL[step] = z[step] * IL[0]  # dimensionalize back to IL
    IL_input[step] = s_2_array[step] * IL[0] # dimensionalize back to IL input

# Save simulation data to CSV file    
with open(f"Output_data/simu_data_test1.csv", mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['t', 'tau', 'x', 'y', 'z', 'E', 'T', 'IL', 's_1', 's_2', 'Fitness'])
    for i in range(num_steps):
        writer.writerow([i, tau[i], x[i], y[i], z[i], E[i], T[i], IL[i], (s_1_array[i]/(t_s*E[0])), (s_2_array[i]/(t_s*IL[0])), fitness_history[i] if i < len(fitness_history) else ''])

print("Simulation completed.")


  0%|          | 0/1999 [00:00<?, ?it/s]

100%|██████████| 1999/1999 [00:00<00:00, 11209.36it/s]

Simulation completed.


**Sweep through array of antigenicity values.**

In [ ]:
#for c_index in range(0, len(c_values)):


## Impliment genetic algorithm

In [3]:
# Dimensional parameters
# Arrow indicates parameters that should be varied for different simulations

# eq.1
c_values = [0, 8.5e-5, 0.0025, 0.006, 0.010, 0.0297, 0.0325, 0.040, 0.050] # antigenicity values to test
c = c_values[2] #0.0297  # antigenicity (keep between 0 and 0.05) <---

mu_2 = 0.03 # Multiplicative inverse of the natural lifetime of effector cells (days^-1)
p_1 = 0.1245 # Proliferation rate of effector cells (days^-1) 
g_1 = 2 * 10**7 # Threshold growth rate of effector cells (cells days^-1)

# eq.2
g_2 = 1 * 10**5 # Threshold for tumor cell removal (cells)
r_2 = 0.18 # tumor growth rate (days^-1) <---
b = 1 * 10**(-9) # Multiplicative inverse of the tumor carrying capacity (cells^-1)
alpha = 1 # Immune system strength for tumor removal (days^-1)

# eq.3
mu_3 = 10 # Multiplicative inverse of the natural lifetime of IL-2 (days^-1)
p_2 = 5.0 # Production rate of IL-2 (units days^-1)
g_3 = 1 * 10**3 # Threshold for IL-2 production via effector-tumor interaction (cells)

# External therapy parameters
s_1 = 0  # external IL-2 source
s_2 = 0  # external T cell source

# Simulation parameters
num_steps = 2000 # number of time steps <---
total_time = 4000  # total time (days) <---

# time arrays
t_s = r_2  # time scale (days) <---
t = np.linspace(0, total_time, int(num_steps/r_2))  # time array
tau = t * t_s  # dimensional time array

# Arrays to hold non-dimensional values
x = np.zeros(num_steps)
y = np.zeros(num_steps)
z = np.zeros(num_steps)
s_1_array = np.zeros(num_steps)
s_2_array = np.zeros(num_steps)

# Arrays to hold population values
E = np.zeros(num_steps)
T = np.zeros(num_steps)
IL = np.zeros(num_steps)
IL_input = np.zeros(num_steps)

# Set initial conditions
E[0], T[0], IL[0] = 1e5, 1e5, 1e2

print("Nondimensionalized parameters:")
print([E[0], T[0], IL[0], t_s, c, p_1, g_1, mu_2, g_2, b, r_2, alpha, mu_3, p_2, g_3, s_1, s_2])

# Nondimensionalize parameters and initial conditions
[c, p_1, g_1, mu_2, g_2, b, r_2, alpha, mu_3, p_2, g_3, s_1, s_2] = nondim(
    E[0], T[0], IL[0], t_s, c, p_1, g_1, mu_2, g_2, b, r_2, alpha, mu_3, p_2, g_3, s_1, s_2)

print("Nondimensionalized parameters:")
print([c, p_1, g_1, mu_2, g_2, b, r_2, alpha, mu_3, p_2, g_3, s_1, s_2])

# Set initial non-dimensional x, y, z values to 1.0
x[0] = E[0] / E[0]
y[0] = T[0] / T[0]
z[0] = IL[0] / IL[0]


Nondimensionalized parameters:
[100000.0, 100000.0, 100.0, 0.18, 0.0025, 0.1245, 20000000, 0.03, 100000, 1e-09, 0.18, 1, 10, 5.0, 1000, 0, 0]
Nondimensionalized parameters:
[0.013888888888888888, 0.6916666666666667, 200000.0, 0.16666666666666666, 1.0, 0.0001, 1.0, 5.555555555555555, 55.55555555555556, 27777.777777777777, 0.01, 0.0, 0.0]


In [4]:
# Genetic Algorithm setup

# Arrays to hold fitness values
fitness = np.zeros(num_steps)
fitness_history = []

# Create instance of GeneticAlgorithm 
ga_func = GeneticAlgorithm()

# Create an instance of the GA class
ga_instance = pygad.GA(num_generations=30,
                       num_parents_mating=10,
                       sol_per_pop=100,
                       num_genes=8,
                       fitness_func=GeneticAlgorithm.fitness_func,
                       on_generation=GeneticAlgorithm.on_generation,
                        keep_parents=-1,
                       save_solutions=False,
                       init_range_low=0.0,
                       init_range_high=1.0,
                       gene_space=[(-10.0, 10.0)]*8,
                       mutation_num_genes=2)

print("Genetic Algorithm initialized.")

Genetic Algorithm initialized.


In [5]:
for step in tqdm.tqdm(range(1, num_steps)):
   
   # Step of Genetic Algorithm for time step
    ga_instance.environment = {'t': step-1, 'x': x[step-1], 'y': y[step-1], 'z': z[step-1]}
    ga_instance.run()
    best_solution, best_solution_fitness, _ = ga_instance.best_solution(pop_fitness=ga_instance.last_generation_fitness)
    fitness_history.append(best_solution_fitness)

    # External input parameters
    genes1 = best_solution[0:4]
    genes2 = best_solution[4:8]

    # Calculate s_1 and s_2 based on genes and current state
    s_1 = genes1[0]*x[step-1] + genes1[1]*y[step-1] + genes1[2]*z[step-1] + genes1[3]
    s_2 = genes2[0]*x[step-1] + genes2[1]*y[step-1] + genes2[2]*z[step-1] + genes2[3]

    # Restrict negative input
    s_1 = max(0, s_1)
    s_2 = max(0, s_2)

    # Store s_1 and s_2
    s_1_array[step] = s_1
    s_2_array[step] = s_2

    # parameters for time step
    params = [c, mu_2, p_1, g_1, s_1_array[step], r_2, b, alpha, g_2, p_2, g_3, mu_3, s_2_array[step]]

    # Solve KP ODEs for time step
    [x[step], y[step], z[step]] = kp_integrate().integrate([x[step-1], y[step-1], z[step-1]], 
                                         params, 
                                         t_span=(tau[step-1], tau[step]))

    E[step] = x[step] * E[0]  # dimensionalize back to E
    T[step] = y[step] * T[0]  # dimensionalize back to T
    IL[step] = z[step] * IL[0]  # dimensionalize back to IL
    IL_input[step] = s_2_array[step] * IL[0] # dimensionalize back to IL input

# Save simulation data to CSV file    
with open(f"Output_data/simu_data_test_ga1.csv", mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['t', 'tau', 'x', 'y', 'z', 'E', 'T', 'IL', 's_1', 's_2', 'Fitness'])
    for i in range(num_steps):
        writer.writerow([i, tau[i], x[i], y[i], z[i], E[i], T[i], IL[i], (s_1_array[i]/(t_s*E[0])), (s_2_array[i]/(t_s*IL[0])), fitness_history[i] if i < len(fitness_history) else ''])

print("Simulation completed.")


100%|██████████| 1999/1999 [00:59<00:00, 33.56it/s]

Simulation completed.


## Antigenicity sweep with GA inputs

In [ ]:
# Different antigenicity values for each run
antigenicity_values = np.linspace(-0.005, 0.05, 100)

# Initialize KP integrator
integrator = kp_integrate()

# Time tracking
start_time = time.time()

# Loop over antigenicity values
for c_s in tqdm.tqdm(antigenicity_values):
    
   # Dimensional parameters
    # Arrow indicates parameters that should be varied for different simulations

    # eq.1
    c = c_s  # antigenicity (keep between 0 and 0.05) <---

    mu_2 = 0.03 # Multiplicative inverse of the natural lifetime of effector cells (days^-1)
    p_1 = 0.1245 # Proliferation rate of effector cells (days^-1) 
    g_1 = 2 * 10**7 # Threshold growth rate of effector cells (cells days^-1)

    # eq.2
    g_2 = 1 * 10**5 # Threshold for tumor cell removal (cells)
    r_2 = 0.18 # tumor growth rate (days^-1) <---
    b = 1 * 10**(-9) # Multiplicative inverse of the tumor carrying capacity (cells^-1)
    alpha = 1 # Immune system strength for tumor removal (days^-1)

    # eq.3
    mu_3 = 10 # Multiplicative inverse of the natural lifetime of IL-2 (days^-1)
    p_2 = 5.0 # Production rate of IL-2 (units days^-1)
    g_3 = 1 * 10**3 # Threshold for IL-2 production via effector-tumor interaction (cells)

    # External therapy parameters
    s_1 = 0  # external IL-2 source
    s_2 = 0  # external T cell source

    # Simulation parameters
    num_steps = 2000 # number of time steps <---
    total_time = 4000  # total time (days) <---

    # time arrays
    t_s = r_2  # time scale (days) <---
    t = np.linspace(0, total_time, int(num_steps/r_2))  # time array
    tau = t * t_s  # dimensional time array

    # Arrays to hold non-dimensional values
    x = np.zeros(num_steps)
    y = np.zeros(num_steps)
    z = np.zeros(num_steps)
    s_1_array = np.zeros(num_steps)
    s_2_array = np.zeros(num_steps)

    # Arrays to hold population values
    E = np.zeros(num_steps)
    T = np.zeros(num_steps)
    IL = np.zeros(num_steps)
    IL_input = np.zeros(num_steps)

    # Set initial conditions
    E[0], T[0], IL[0] = 1e5, 1e5, 1e2

    # Nondimensionalize parameters and initial conditions
    [c, p_1, g_1, mu_2, g_2, b, r_2, alpha, mu_3, p_2, g_3, s_1, s_2] = nondim(
        E[0], T[0], IL[0], t_s, c, p_1, g_1, mu_2, g_2, b, r_2, alpha, mu_3, p_2, g_3, s_1, s_2)

    # Set initial non-dimensional x, y, z values to 1.0
    x[0] = E[0] / E[0]
    y[0] = T[0] / T[0]
    z[0] = IL[0] / IL[0]

    # Genetic Algorithm setup

    # Arrays to hold fitness values
    fitness = np.zeros(num_steps)
    fitness_history = []

    # Create instance of GeneticAlgorithm 
    ga_func = GeneticAlgorithm()

    # Create an instance of the GA class
    ga_instance = pygad.GA(num_generations=30,
                        num_parents_mating=10,
                        sol_per_pop=100,
                        num_genes=8,
                        fitness_func=GeneticAlgorithm.fitness_func,
                        on_generation=GeneticAlgorithm.on_generation,
                            keep_parents=-1,
                        save_solutions=False,
                        init_range_low=0.0,
                        init_range_high=1.0,
                        gene_space=[(-10.0, 10.0)]*8,
                        mutation_num_genes=2)

    # Run simulation over time steps
    for step in (range(1, num_steps)):
        
        # Step of Genetic Algorithm for time step
        ga_instance.environment = {'t': step-1, 'x': x[step-1], 'y': y[step-1], 'z': z[step-1]}
        ga_instance.run()
        best_solution, best_solution_fitness, _ = ga_instance.best_solution(pop_fitness=ga_instance.last_generation_fitness)
        fitness_history.append(best_solution_fitness)

        # External input parameters
        s_1_array[step] = best_solution[0]
        s_2_array[step] = best_solution[1]

        # parameters for time step
        params = [c, mu_2, p_1, g_1, s_1_array[step], r_2, b, alpha, g_2, p_2, g_3, mu_3, s_2_array[step]]

        # Solve KP ODEs for time step
        [x[step], y[step], z[step]] = integrator.integrate([x[step-1], y[step-1], z[step-1]], 
                                            params, 
                                            t_span=(tau[step-1], tau[step]))

        E[step] = x[step] * E[0]  # dimensionalize back to E
        T[step] = y[step] * T[0]  # dimensionalize back to T
        IL[step] = z[step] * IL[0]  # dimensionalize back to IL
        IL_input[step] = s_2_array[step] * IL[0] # dimensionalize back to IL input
    
    with open(f"Output_data/0003_c_sweep_ga_0/simu_data_0003_c_ga_{c_s+1}_.csv", mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['t', 'tau', 'x', 'y', 'z', 'E', 'T', 'IL', 's_1', 's_2', 'Fitness'])
        
        for i in range(num_steps):
            writer.writerow([i, tau[i], x[i], y[i], z[i], E[i], T[i], IL[i], (s_1_array[i]/(t_s*E[0])), (s_2_array[i]/(t_s*IL[0])), 
                             fitness_history[i] if i < len(fitness_history) else ''])    
    print("Saved as", file)

print("Simulation completed.\n")

# Time tracking
end_time = time.time()
print(f"Total simulation time: {(end_time - start_time)/60} minutes")

  1%|          | 1/100 [00:56<1:32:36, 56.13s/it]

Saved as <_io.TextIOWrapper name='Output_data/0003_c_sweep_ga_0/simu_data_0003_c_ga_0.995_.csv' mode='w' encoding='UTF-8'>


  2%|▏         | 2/100 [01:53<1:32:54, 56.89s/it]

Saved as <_io.TextIOWrapper name='Output_data/0003_c_sweep_ga_0/simu_data_0003_c_ga_0.9955555555555555_.csv' mode='w' encoding='UTF-8'>


  3%|▎         | 3/100 [02:50<1:32:20, 57.12s/it]

...

Saved as <_io.TextIOWrapper name='Output_data/0003_c_sweep_ga_0/simu_data_0003_c_ga_0.9983333333333333_.csv' mode='w' encoding='UTF-8'>


  8%|▊         | 8/100 [07:41<1:29:18, 58.24s/it]

Saved as <_io.TextIOWrapper name='Output_data/0003_c_sweep_ga_0/simu_data_0003_c_ga_0.9988888888888889_.csv' mode='w' encoding='UTF-8'>


  9%|▉         | 9/100 [08:40<1:28:24, 58.29s/it]

Saved as <_io.TextIOWrapper name='Output_data/0003_c_sweep_ga_0/simu_data_0003_c_ga_0.9994444444444445_.csv' mode='w' encoding='UTF-8'>
